# Looping Through different variables to find the best combination

## Results
### Starting Data - 1 month
- Best Results
    1. Data Order =
    2. Underlying Data Type =
    3. Number of Feilds =
    4. Derivatives =
    5. TA's =
- Best Combination for XGBoost
    1. Data Order =
    2. Underlying Data Type =
    3. Number of Feilds =
    4. Derivatives =
    5. TA's =
- Best Combination for Logistic Regression
    1. Data Order =
    2. Underlying Data Type =
    3. Number of Feilds =
    4. Derivatives =
    5. TA's = 
- Best Combination for LSTM
    1. Data Order =
    2. Underlying Data Type =
    3. Number of Feilds =
    4. Derivatives =
    5. TA's =

## Data
1. Data Order - Either the data is in the original order or the data is randomized after being turned into packets. This is because I believe randomizing the data will prevent overfitting better than it being in a predicable order, because I do not believe the packets should be linked, for best results they should be seen as independent by the data.
2. Underlying Data Type - Either the data is Raw or a percent change. I am not sure whether using the raw data or a decimal change of the number is better for the derivatives.
3. Number of fields - This is the number of look back periods in each packet
4. Derivatives - Using the first derivative for ROC and the second derivative for acceleration, on both the raw data as well as different TA's
5. TA's - Trend analysis that I want to use, for now we are using different EMA's and SMA's plus possibly RSI14 and Bolinger Bands, to determine if any of them will enhance the outcome

## Process
### Get the Data
1. Start with 1 month of data for speed
2. Continue with 1 year of data
3. Based on the random data vs ordered data, take 1 year of random data
    (Look at the the whole database and pull out a number of random entries equivalent to 1 years worth)
4. Take the most efficient process and apply it to the whole dataset in subgroups, randomly assigning groups. Steps 3 and 4 are to prevent overfitting over a single Ticker

### Processing 
1. I have 7 variables to test
    1. Number of Fields - Number of fields in the packet (lookback period)
    2. Derivative - Using the First and Second Derivative, and using it on:
        - Raw Data
        - Percent Change
        - EMA8, SMA8, EMA20, SMA20
        - Mix of these
    3. EMAs
    4. SMAs
    5. Other
    6. Underlying Data
    7. Data Order
2. Loop Through all of the variables and test the XGBoost on all of them
3. Loop Through all of them and test Logistic Regression
4. Loop Through all of them and test LSTM Neural Network


# Getting Data

In [13]:
import duckdb
import pandas as pd
import pandas_ta as ta
import os
import csv

con = duckdb.connect(r"C:\Users\rybot\OneDrive\Databases\datadump.duckdb")

qry = '''
    SELECT * FROM Stocks
    WHERE EXTRACT(YEAR FROM Datetime) = 2023 
    AND Stock = 'AAPL' 
    AND Interval = 1
'''

qry_month = '''
    SELECT * FROM Stocks
    WHERE Datetime >= '2023-03-01' 
    AND Datetime < '2023-04-01' 
    AND Stock = 'AAPL' 
    AND Interval = 1
'''

qry_all = '''
    SELECT * FROM Stocks
    WHERE Stock = 'AAPL' 
    AND Interval = 1
'''

df = con.execute(qry_all).fetchdf()
# Sort the Data from oldest to newest
df = df.sort_values(by='Datetime').reset_index(drop=True)


print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2562781 entries, 0 to 2562780
Data columns (total 8 columns):
 #   Column    Dtype         
---  ------    -----         
 0   Datetime  datetime64[us]
 1   open      float32       
 2   high      float32       
 3   low       float32       
 4   close     float32       
 5   volume    float32       
 6   Stock     object        
 7   interval  float32       
dtypes: datetime64[us](1), float32(6), object(1)
memory usage: 97.8+ MB
None


#### Selling Function

In [9]:
def trail_stoploss(row, tradingdata, whole_percentage):
    idx = row.name
    td = tradingdata
    pct = whole_percentage/100
    max_holdtime = 30 # Only hold the trade for twenty minutes

    after_td = td.loc[idx:idx+max_holdtime+1] 
    
    basis = td['close'].loc[idx]
    max_price = basis   # Initially the max price is the basis - No Shorting to Start
    min_price = basis - (basis*pct)
    time_stop = idx + max_holdtime 
    time_counter = idx
    
    #print(f"Basis: {basis}")

    for i, row in after_td.iterrows():
        close = row['close']
        #print(f"Close: {close}, Max: {max_price}, Min: {min_price}, Counter: {time_counter-idx}")
        if close > max_price:
            max_price = close
            min_price = max_price - (max_price * pct)
        elif close < min_price:
            profit = (close - basis) / basis
            return [i, profit]
        
        time_counter += 1
        if time_counter > time_stop:
            break

    # If the loop ends without triggering stop loss, calculate the profit based on the last close
    profit = (close - basis) / basis
    return [i, profit]

#### Trading Data

In [ ]:
import time

time_overall = time.time()

# Getting all of the trading days because trades can only occur during trading day
df['Time'] = df['Datetime'].dt.time
start_time = pd.to_datetime('09:00:00').time()
end_time = pd.to_datetime('16:00:00').time()
TradingDay_df = df[(df['Time'] > start_time) & (df['Time'] < end_time) & (df.index >= 201)]
TradingDay_df['Profit'] = TradingDay_df.apply(trail_stoploss, axis=1, tradingdata=TradingDay_df,whole_percentage=5)

t2 = time.time()

print(f"Finished calculating Profit in {t2 - time_overall} seconds")

#applying EMAs SMAs
EMAs = [8,10,20,50,75,100]
SMAs = [8,10,20,50,75,100]
Other_tas = ["RSI14","Bolinger Bands"]


start_index = max(0,df.index.get_loc(TradingDay_df.index[0]) - 201)
end_index = start_index + len(TradingDay_df) + 202  
sub_df = df.iloc[start_index:end_index]

for EMA in EMAs:
    sub_df[f'EMA{EMA}'] = ta.ema(sub_df['close'], length=EMA)
    df.loc[sub_df.index, f'EMA{EMA}'] = sub_df[f'EMA{EMA}']

t3 = time.time()

print(f"Finished calculating EMAs in {t3 - t2} seconds")

for SMA in SMAs:
    sub_df[f'SMA{SMA}'] = ta.sma(sub_df['close'], length=SMA)
    df.loc[sub_df.index, f'SMA{SMA}'] = sub_df[f'SMA{SMA}']

t4 = time.time()

print(f"Finished calculating SMAs in {t4 - t2} seconds")

# Calculating 1st 2nd Deritivatives
# 1st
sub_df['Time_Diff'] = sub_df['Datetime'].diff().dt.total_seconds()
sub_df['dydx'] = sub_df['close'].diff() / sub_df['Time_Diff']
df.loc[sub_df.index, 'dydx'] = sub_df['dydx']

t5 = time.time()

print(f"Finished calculating 1st Derivative in {t5 - t4} seconds")

# 2nd
sub_df['dydx2'] = sub_df['dydx'].diff() / sub_df['Time_Diff']
df.loc[sub_df.index, 'dydx2'] = sub_df['dydx2']


time_end = time.time()

print(f"Finished calculating 2nd Derivative in {time_end - t5} seconds")
print(f"time taken for complete data processing = {time_end - time_overall}")

time_save = time.time()

df.to_csv('C:/Users/rybot/OneDrive/Databases/Processed_Data090324.csv', index=True)
TradingDay_df.to_csv('C:/Users/rybot/OneDrive/Databases/Processed_TradingData090324.csv', index=True)

time_savend = time.time()
print(f"Finished saving the data in f{time_savend-time_save} seconds")


In [3]:
import duckdb
import pandas as pd
import pandas_ta as ta
import os
import csv


df = pd.read_csv('C:/Users/rybot/OneDrive/Databases/Processed_Data090324.csv')
TradingDay_df = pd.read_csv('C:/Users/rybot/OneDrive/Databases/Processed_TradingData090324.csv')

In [4]:
# List of Testing Variables
Data_order = ["Random","Sorted"]
Underlying_data = ["Percent Change","Raw Data"]
Field_size = [10,20,50,100,200,300]
Derivatives = ["None","First",["First","Second"]]
TAs = [["None"],["EMA"],["EMA","SMA"],["EMA","Other"]]

# Processing for XGBoost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

params = { # From the Grid Search Previously Done
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'colsample_bytree': 0.6,
    'gamma': 0,
    'learning_rate': 0.2,
    'max_depth': 10,
    'min_child_weight': 3,
    'n_estimators': 300,
    'subsample': 0.8
}

# Variables -  Field size, Derivatives, TAs, Data_order




        


        

# Processing for Random Forest

# Processing for LSTM 